<a href="https://colab.research.google.com/github/SpringBoardMentor193s/SentinelNet_Oct_Batch/blob/Aryan_Sahu/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
column_names = [
'duration', 'protocol_type', 'service', 'flag',
'src_bytes', 'dst_bytes',
'land', 'wrong_fragment', 'urgent', 'hot',
'num_failed_logins',
'logged_in', 'num_compromised',
'root_shell', 'su_attempted',
'num_root', 'num_file_creations',
'num_shells', 'num_access_files',
'num_outbound_cmds', 'is_host_login',
'is_guest_login', 'count',
'srv_count', 'serror_rate', 'srv_serror_rate',
'rerror_rate',
'srv_rerror_rate', 'same_srv_rate',
'diff_srv_rate', 'srv_diff_host_rate',
'dst_host_count', 'dst_host_srv_count',
'dst_host_same_srv_rate',
'dst_host_diff_srv_rate',
'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate',
'dst_host_serror_rate',
'dst_host_srv_serror_rate',
'dst_host_rerror_rate',
'dst_host_srv_rerror_rate', 'class', 'difficulty' ]

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
